In [672]:
from pathlib import Path
import numpy as np

In [673]:
input_path = "example.txt"
input = Path(input_path).read_text()
print(input)

##########
#..O..O.O#
#......O.#
#.OO..O.O#
#..O@..O.#
#O#..O...#
#O..O..O.#
#.OO.O.OO#
#....O...#
##########

<vv>^<v^>v>^vv^v>v<>v^v<v<^vv<<<^><<><>>v<vvv<>^v^>^<<<><<v<<<v^vv^v>^
vvv<<^>^v^^><<>>><>^<<><^vv^^<>vvv<>><^^v>^>vv<>v<<<<v<^v>^<^^>>>^<v<v
><>vv>v^v^<>><>>>><^^>vv>v<^^^>>v^v^<^^>v^^>v^<^v>v<>>v^v^<v>v^^<^^vv<
<<v<^>>^^^^>>>v^<>vvv^><v<<<>^^^vv^<vvv>^>v<^^^^v<>^>vvvv><>>v^<<^^^^^
^><^><>>><>^^<<^^v>>><^<v>^<vv>>v>>>^v><>^v><<<<v>>v<v<v>vvv>^<><<>^><
^>><>^v<><^vvv<^^<><v<<<<<><^v<<<><<<^^<v<^^^><^>>^<v^><<<^>>^v<v^v<v^
>^>>^v>vv>^<<^v<>><<><<v<<v><>v<^vv<<<>^^v^>^^>>><<^v>>v^v><^^>>^<>vv^
<><^^>^^^<><vvvvv^v<v<<>^v<v>v<<^><<><<><<<^^<<<^<<>><<><^^^>^^<>^>v<>
^^>vv<^v^v<vv>^<><v<^v>^^^>>>^^vvv^>vvv<>>>^<^>>>>>^<<^v>^vvv<>^<><<v>
v^^>>><<^^<>>^v^<v^vv<>v^<<>^<^v^v><^<<<><<^<v><v<>vv>>v><v^<vv<>v^<<^



## Part I

In [674]:
EMPTY = 0
ROBOT = 1
BOX = 2
WALL = 3

entity_map = {
    "@": ROBOT,
    "O": BOX,
    "#": WALL,
    ".": EMPTY,
}

direction_map = {
    "^": np.array([-1, 0]),
    ">": np.array([0, 1]),
    "v": np.array([1, 0]),
    "<": np.array([0, -1]),
}

In [675]:
warehouse_string, directions_string = input.split("\n\n")

In [676]:
warehouse = np.array(
    [
        [entity_map[character] for character in line]
        for line in warehouse_string.splitlines()
    ],
    dtype=np.int8,
)
print(warehouse)

[[3 3 3 3 3 3 3 3 3 3]
 [3 0 0 2 0 0 2 0 2 3]
 [3 0 0 0 0 0 0 2 0 3]
 [3 0 2 2 0 0 2 0 2 3]
 [3 0 0 2 1 0 0 2 0 3]
 [3 2 3 0 0 2 0 0 0 3]
 [3 2 0 0 2 0 0 2 0 3]
 [3 0 2 2 0 2 0 2 2 3]
 [3 0 0 0 0 2 0 0 0 3]
 [3 3 3 3 3 3 3 3 3 3]]


In [677]:
directions = [
    direction_map[direction_character]
    for direction_character in directions_string.replace("\n", "")
]

In [678]:
def move(old_position, direction, type) -> np.ndarray:
    attempt_position = old_position + direction
    if warehouse[tuple(attempt_position)] == WALL:
        return False
    if warehouse[tuple(attempt_position)] == EMPTY:
        warehouse[tuple(old_position)] = EMPTY
        warehouse[tuple(attempt_position)] = type
        return True
    if warehouse[tuple(attempt_position)] == BOX:
        if move(attempt_position, direction, BOX):
            warehouse[tuple(old_position)] = EMPTY
            warehouse[tuple(attempt_position)] = type
            return True
        else:
            return False


for direction in directions:
    robot_position = np.argwhere(warehouse == ROBOT)[0]
    move(robot_position, direction, ROBOT)

In [679]:
print(warehouse)

[[3 3 3 3 3 3 3 3 3 3]
 [3 0 2 0 2 0 2 2 2 3]
 [3 0 0 0 0 0 0 0 0 3]
 [3 2 2 0 0 0 0 0 0 3]
 [3 2 2 1 0 0 0 0 0 3]
 [3 2 3 0 0 0 0 0 2 3]
 [3 2 0 0 0 0 0 2 2 3]
 [3 2 0 0 0 0 0 2 2 3]
 [3 2 2 0 0 0 0 2 2 3]
 [3 3 3 3 3 3 3 3 3 3]]


In [680]:
box_positions = np.argwhere(warehouse == BOX)
gps_coords = box_positions * np.array([100, 1])
np.sum(gps_coords)

10092

## Part II

In [681]:
# widening map
EMPTY = 0
ROBOT = 1
BOX = 2
WALL = 3
LEFT_BOX = 8
RIGHT_BOX = 9

widening_map = {
    ".": "..",
    "@": "@.",
    "O": "[]",
    "#": "##",
}

entity_map = {
    "@": ROBOT,
    "[": LEFT_BOX,
    "]": RIGHT_BOX,
    "#": WALL,
    ".": EMPTY,
}


wide_warehouse_string = warehouse_string
for entity, wide_entity in widening_map.items():
    wide_warehouse_string = wide_warehouse_string.replace(entity, wide_entity)
print(wide_warehouse_string)

####################
##....[]....[]..[]##
##............[]..##
##..[][]....[]..[]##
##....[]@.....[]..##
##[]##....[]......##
##[]....[]....[]..##
##..[][]..[]..[][]##
##........[]......##
####################


In [ ]:
wide_warehouse = np.array(
    [
        [entity_map[character] for character in line]
        for line in wide_warehouse_string.splitlines()
    ],
    dtype=np.int8,
)
print(wide_warehouse)

In [682]:
def check_move(from_position: np.ndarray, direction: np.ndarray) -> bool:
    to_position = from_position + direction
    in_way = wide_warehouse[tuple(to_position)]
    if in_way == EMPTY:
        return True

    if in_way == WALL:
        return False

    if (direction == direction_map["^"]).all() or (
        direction == direction_map["v"]
    ).all():
        if in_way == LEFT_BOX:
            right_box = check_move(to_position + direction_map[">"], direction)
            left_box = check_move(to_position, direction)
        elif in_way == RIGHT_BOX:
            right_box = check_move(to_position, direction)
            left_box = check_move(to_position + direction_map["<"], direction)
        return right_box and left_box

    if (direction == direction_map[">"]).all() or (
        direction == direction_map["<"]
    ).all():
        return check_move(to_position, direction)

    raise Exception


def do_move(from_position: np.ndarray, direction: np.ndarray) -> None:
    to_position = from_position + direction
    in_way = wide_warehouse[tuple(to_position)]
    
    if in_way == WALL:
        raise Exception
    
    if in_way == EMPTY:
        entity = wide_warehouse[tuple(from_position)]
        wide_warehouse[tuple(from_position)] = EMPTY
        wide_warehouse[tuple(to_position)] = entity
        return

    if (direction == direction_map["^"]).all() or (
        direction == direction_map["v"]
    ).all():
        if in_way == LEFT_BOX:
            do_move(to_position + direction_map[">"], direction)
            do_move(to_position, direction)
            do_move(from_position, direction)
        elif in_way == RIGHT_BOX:
            do_move(to_position + direction_map["<"], direction)
            do_move(to_position, direction)
            do_move(from_position, direction)
        else:
            raise Exception
        return

    if (direction == direction_map[">"]).all() or (
        direction == direction_map["<"]
    ).all():
        do_move(to_position, direction)
        do_move(from_position, direction)

In [683]:
for direction in directions[:]:
    robot_position = np.argwhere(wide_warehouse == ROBOT)[0]
    if check_move(robot_position, direction):
        do_move(robot_position, direction)

print(wide_warehouse)

[[3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3]
 [3 3 8 9 0 0 0 0 0 0 0 8 9 0 8 9 8 9 3 3]
 [3 3 8 9 0 0 0 0 0 0 0 0 0 0 0 8 9 0 3 3]
 [3 3 8 9 0 0 0 0 0 0 0 0 8 9 8 9 8 9 3 3]
 [3 3 8 9 0 0 0 0 0 0 8 9 0 0 0 0 8 9 3 3]
 [3 3 0 0 3 3 0 0 0 0 0 0 8 9 0 0 0 0 3 3]
 [3 3 0 0 8 9 0 0 0 0 0 0 0 0 0 0 0 0 3 3]
 [3 3 0 0 1 0 0 0 0 0 0 8 9 0 8 9 8 9 3 3]
 [3 3 0 0 0 0 0 0 8 9 8 9 0 0 8 9 0 0 3 3]
 [3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3]]


In [684]:
box_positions = np.argwhere(wide_warehouse == LEFT_BOX)
gps_coords = box_positions * np.array([100, 1])
np.sum(gps_coords)

9021